In [0]:
dbutils.fs.mount(source = "wasbs://week18@wee18strg.blob.core.windows.net", mount_point = "/mnt/week18/raw", extra_configs = {"fs.azure.account.key.wee18strg.blob.core.windows.net":"ntlAI5omvmq6GSWKCHGvXTQioREicpBccmckE2xEinQ95xAUZWIhfDrl9uicE/nyTI83nnh9OCUn+AStOq9FWw=="})

True

In [0]:
%fs ls dbfs:/databricks-datasets/wine-quality/

path,name,size,modificationTime
dbfs:/databricks-datasets/wine-quality/README.md,README.md,1066,1594264657000
dbfs:/databricks-datasets/wine-quality/winequality-red.csv,winequality-red.csv,84199,1594264658000
dbfs:/databricks-datasets/wine-quality/winequality-white.csv,winequality-white.csv,264426,1594264658000


In [0]:
red_df = spark.read.format("csv").option("header", "true").load("dbfs:/databricks-datasets/wine-quality/winequality-red.csv")
red_df.show(2, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5                                                                                                                         |
|7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
red_df =  spark.read.text("dbfs:/databricks-datasets/wine-quality/winequality-red.csv")
header = red_df.first()[0]
print(header)
filtered =  red_df.filter(red_df.value != header)
filtered.show(2)
schema = header.split(";")
print(schema)
final_red_df = filtered.rdd.map(lambda x : x[0].split(";")).toDF(schema)
final_red_df.show(2, False)

"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
+--------------------+
|               value|
+--------------------+
|7.4;0.7;0;1.9;0.0...|
|7.8;0.88;0;2.6;0....|
+--------------------+
only showing top 2 rows

['"fixed acidity"', '"volatile acidity"', '"citric acid"', '"residual sugar"', '"chlorides"', '"free sulfur dioxide"', '"total sulfur dioxide"', '"density"', '"pH"', '"sulphates"', '"alcohol"', '"quality"']
+---------------+------------------+-------------+----------------+-----------+---------------------+----------------------+---------+----+-----------+---------+---------+
|"fixed acidity"|"volatile acidity"|"citric acid"|"residual sugar"|"chlorides"|"free sulfur dioxide"|"total sulfur dioxide"|"density"|"pH"|"sulphates"|"alcohol"|"quality"|
+---------------+------------------+-------------+----------------+-----------+---------------------+---------------

In [0]:
white_df = spark.read.text("dbfs:/databricks-datasets/wine-quality/winequality-white.csv")
header = white_df.first()[0]
print(header)
white_filtered = white_df.filter(white_df.value != header)
white_filtered.show(2)
schema = header.split(";")
white_df = white_filtered.rdd.map(lambda x : x[0].split(";")).toDF(schema)
white_df.show()

"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
+--------------------+
|               value|
+--------------------+
|7;0.27;0.36;20.7;...|
|6.3;0.3;0.34;1.6;...|
+--------------------+
only showing top 2 rows

+---------------+------------------+-------------+----------------+-----------+---------------------+----------------------+---------+----+-----------+---------+---------+
|"fixed acidity"|"volatile acidity"|"citric acid"|"residual sugar"|"chlorides"|"free sulfur dioxide"|"total sulfur dioxide"|"density"|"pH"|"sulphates"|"alcohol"|"quality"|
+---------------+------------------+-------------+----------------+-----------+---------------------+----------------------+---------+----+-----------+---------+---------+
|              7|              0.27|         0.36|            20.7|      0.045|                   45|                   170|    1.001|   3|       0.45|

In [0]:
print(schema)

['"fixed acidity"', '"volatile acidity"', '"citric acid"', '"residual sugar"', '"chlorides"', '"free sulfur dioxide"', '"total sulfur dioxide"', '"density"', '"pH"', '"sulphates"', '"alcohol"', '"quality"']


In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, FloatType, StringType
from pyspark.sql.functions import from_unixtime, expr,row_number, lit
from pyspark.sql import Window
wine_schema = StructType([
    StructField("fixed_acidity", FloatType(), True),
    StructField("volatile_acidity", FloatType(), True),
    StructField("citric_acid", FloatType(), True),
    StructField("residual_sugar", FloatType(), True),
    StructField("chlorides", FloatType(), True),
    StructField("free_sulfur_dioxide", FloatType(), True),
    StructField("total_sulfur_dioxide", FloatType(), True),
    StructField("density", FloatType(), True),
    StructField("pH", FloatType(), True),
    StructField("sulphates", FloatType(), True),
    StructField("alchol", FloatType(), True),
    StructField("quantity", FloatType(), True),
])

In [0]:
white_df.write.format("csv").mode("append").save("/mnt/week18/raw")

In [0]:
final_red_df.write.format("csv").mode("append").save("/mnt/week18/raw")

In [0]:
wine_df = spark.read.format("csv").schema(wine_schema).load("/mnt/week18/raw")
wine_df.show(2,False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+------+--------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|pH |sulphates|alchol|quantity|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+------+--------+
|7.0          |0.27            |0.36       |20.7          |0.045    |45.0               |170.0               |1.001  |3.0|0.45     |8.8   |6.0     |
|6.3          |0.3             |0.34       |1.6           |0.049    |14.0               |132.0               |0.994  |3.3|0.49     |9.5   |6.0     |
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+------+--------+
only showing top 2 rows



In [0]:
wine_df = wine_df.withColumn("idx", row_number().over(Window.orderBy(lit(1))))
wine_df.show(2,False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+------+--------+---+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|pH |sulphates|alchol|quantity|idx|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+------+--------+---+
|7.0          |0.27            |0.36       |20.7          |0.045    |45.0               |170.0               |1.001  |3.0|0.45     |8.8   |6.0     |1  |
|6.3          |0.3             |0.34       |1.6           |0.049    |14.0               |132.0               |0.994  |3.3|0.49     |9.5   |6.0     |2  |
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+------+--------+---+
only showing top 2 rows



In [0]:
%sql

create database if not exists wine_db;

In [0]:
wine_df.write.format("parquet").mode("overwrite").save("/mnt/week18/raw/wine/parquet")

In [0]:
%sql

create table if not exists wine_db.wine using parquet location "dbfs:/mnt/week18/raw/wine/parquet"

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [UC_FILE_SCHEME_FOR_TABLE_CREATION_NOT_SUPPORTED] Creating table in Unity Catalog with file scheme dbfs is not supported.
Instead, please create a federated data source connection using the CREATE CONNECTION command for the same table provider, then create a catalog based on the connection with a CREATE FOREIGN CATALOG command to reference the tables therein. SQLSTATE: 0AKUC
	at com.databricks.unity.CredentialScopeSQLHelper$.checkPathOperations(CredentialScopeSQLHelper.scala:100)
	at com.databricks.unity.CredentialScopeSQLHelper$.register(CredentialScopeSQLHelper.scala:151)
	at com.databricks.unity.CredentialScopeSQLHelper$.registerCreateTableAccess(CredentialScopeSQLHelper.scala:592)
	at com.databricks.sql.managedcatalog.CredentialScopeTableCredentialHandler.injectCredential(ResolveWithCredential.scala:487)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.com$databricks$sql$managedcatalog$ResolveWithCredential$$maybeDecorateCatalogTable(ResolveWithCredential.scala:78)
	at com.databricks.sql.managedcatalog.ResolveWithCredential$$anonfun$apply$1.applyOrElse(ResolveWithCredential.scala:151)
	at com.databricks.sql.managedcatalog.ResolveWithCredential$$anonfun$apply$1.applyOrElse(ResolveWithCredential.scala:149)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:377)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:217)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:213)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:39)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:208)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:39)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.apply(ResolveWithCredential.scala:149)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.apply(ResolveWithCredential.scala:64)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$4(RuleExecutor.scala:309)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$3(RuleExecutor.scala:309)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:91)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:306)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeBatch$1(RuleExecutor.scala:289)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$8(RuleExecutor.scala:378)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$8$adapted(RuleExecutor.scala:378)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:378)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.r